In [21]:
# import internal .py modules
import file_path_management as fpath
import public_library as plib

In [22]:
# import packages
import csv
import pandas as pd
import PyPDF2
import requests
import time
import os
import random
from requests.auth import HTTPProxyAuth
from bs4 import BeautifulSoup
import numpy as np
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchElementException
import os
import re
from lxml import etree

In [23]:
websites = [
    'ncbi.nlm.nih.gov', 'elsevier.com', 'wiley.com', 'springer.com', 'physiology.org', 'oup.com', 
    'cambridge.org', 'karger.com', 'lww.com', 'nature.com', 'science.org', 'tandfonline.com', 
    'sagepub.com', 'jamanetwork.com', 'neurology.org', 'biorxiv.org', 'royalsocietypublishing.org', 
    'psycnet.apa.org', 'arvojournals.org', 'jstage.jst.go.jp', 'psychiatryonline.org', 'europepmc.org', 
    'mit.edu', 'thejns.org', 'annualreviews.org', 'snmjournals.org', 'aspetjournals.org', 'elibrary.ru', 
    'books.google.de', 'architalbiol.org', 'ahajournals.org', 'liebertpub.com', 'acs.org', 'degruyter.com', 
    'worldscientific.com', 'iospress.com', 'asahq.org', 'thieme-connect.de', 'neurologia.com', 'mpg.de', 
    'opg.optica.org', 'mcgill.ca', 'rbojournal.org', 'taylorfrancis.com', 'ekja.org', 'www.imrpress.com', 
    'theses.fr', 'ieee.org', 'ssrn.com', 'open.bu.edu', 'journals.biologists.com', 'aip.org', 'lib.wfu.edu', 
    'literatumonline.com', 'scholarpedia.org', 'isho.jp', 'mirasmart.com', 'microbiologyresearch.org', 
    'aps.org', 'ujms.net', 'www.ingentaconnect.com', 'symposium.cshlp.org', 'cabdirect.org', 'ajtmh.org'
]

In [24]:
# cambridge.org
def func_cambridge_org(url):
    # initialize
    info = {
        "doi": np.nan,
        "pmid": np.nan,
        "pmcid": np.nan,
        "title": np.nan,
        "abstract": np.nan,
        "keywords": np.nan,
        "introduction": np.nan,
        "pdf_link": np.nan
    }

    # set up the webdriver
    os.environ['WDM_LOG'] = '0'
    options = Options()
    options.add_argument('--headless')
    driver = webdriver.Chrome()

    # load the webpage
    error_label = 0
    while(error_label == 0):
        try:
            driver.get(url)
            time.sleep(3)
            error_label = 1
        except:
            print("Extracting content from:" + url + " failed, retrying... This might take longer than 5 minutes...")
            time.sleep(5*60)
            error_label = 0
    
    # doi, pmid, pmcid
    doi = driver.find_element(By.XPATH, "//dl[@class='article-details']").find_element(By.XPATH, "//div[@class='doi-data']").find_element(By.XPATH, "//span[@class='text']").text
    print(doi)
    # .split("doi.org/")[1]
    try:
        doi = driver.find_element(By.XPATH, "//div[@class='doi-data')]").find_element(By.XPATH, "//a[@class='app-link app-link__text app-link--accent'])").find_element(By.XPATH, "//span[@class='text'])").text.split("doi.org/")[1]
        doi = doi.strip()
    except:
        doi = np.nan
    pmid = np.nan
    pmcid = np.nan

    # title
    try:
        title = driver.find_element(By.TAG_NAME, "h1").text
        title = title.strip()
    except:
        title = np.nan
    
    # abstract
    try:
        abstract = ""
        elems = driver.find_element(By.XPATH, "//div[@class='abstract']").find_elements(By.TAG_NAME, 'p')
        for elem in elems:
            abstract = abstract + elem.text + " "
        abstract = abstract.strip()
    except:
        abstract = np.nan
    
    # keywords
    try:
        keywords = ""
        elems = driver.find_element(By.XPATH, "//div[@class='keywords']").find_element(By.XPATH, "//div[@class='row keywords__pills']").find_elements(By.TAG_NAME, 'a')
        for elem in elems:
            keywords = keywords + elem.find_element(By.TAG_NAME, 'span').text + "; "
        keywords = keywords.strip()
    except:
        keywords = np.nan
    
    # # introduction
    # try:
    #     intro = ""
    #     elements = driver.find_elements(By.TAG_NAME, "h2")
    #     for element in elements:
    #         if "Introduction" in element.text:
    #             ele_paren = element.find_element(By.XPATH, "..")
    #             intros = ele_paren.find_elements(By.TAG_NAME, "p")
    #             for intro_ele in intros:
    #                 intro = intro + intro_ele.text + " "
    #             break
    #     intro = intro.strip()
    # except:
    #     intro = np.nan
    intro = np.nan

    # pdf_link
    # try:
    #     pdf_link = driver.find_element(By.XPATH, "//a[@class='al-link pdf article-pdfLink']").get_attribute('href')
    #     pdf_link = pdf_link.strip()
    # except:
    #     pdf_link = np.nan
    pdf_link = np.nan

    driver.quit()

    info = {
        "doi": doi,
        "pmid": pmid,
        "pmcid": pmcid,
        "title": title,
        "abstract": abstract,
        "keywords": keywords,
        "introduction": intro,
        "pdf_link": pdf_link
    }

    return info
# --------------------start of test code--------------------
# url = "https://www.cambridge.org/core/journals/visual-neuroscience/article/abs/preoccipital-cortex-receives-a-differential-input-from-the-frontal-eye-field-and-projects-to-the-pretectal-olivary-nucleus-and-other-visuomotorrelated-structures-in-the-rhesus-monkey/2635A959CFED4F09B23B74C0F21CB177"
url = "https://www.cambridge.org/core/journals/thalamus-and-related-systems/article/abs/pathways-for-emotions-and-memory-i-input-and-output-zones-linking-the-anterior-thalamic-nuclei-with-prefrontal-cortices-in-the-rhesus-monkey/82C8F1BECABA367B0FBCD7E3EAACAAB3"
info = func_cambridge_org(url)
print(info["doi"])
print(info["pmid"])
print(info["pmcid"])
print(info["title"])
print(info["abstract"])
print(info["keywords"])
print(info["introduction"])
print(info["pdf_link"])
# ---------------------end of test code---------------------


nan
nan
nan
Pathways for emotions and memory I. Input and output zones linking the anterior thalamic nuclei with prefrontal cortices in the rhesus monkey
The anterior thalamic nuclei occupy a central position in pathways associated with emotions and memory [AMA Arch. Neurol. Psychiatry 38 (1937) 725]. The goal of this study was to determine the anatomic interaction of the anterior nuclei with distinct prefrontal cortices that have been implicated in emotion and specific aspects of memory. To address this issue, we investigated the relationship of input and output zones in the anterior thalamic nuclei linking them with functionally distinct orbitofrontal, medial, and lateral prefrontal cortices. We identified input zones by mapping the pattern and topography of terminations of prefrontal axons, and the output zones by mapping projection neurons in the anterior nuclei, after injection of anterograde and bidirectional tracers in distinct prefrontal cortices. The results showed that the a